<a name='1'></a>
## Import Packages and load data

In [1]:
### -*- coding: utf-8 -*-
"""
Created on Mon May 24 13:26:13 2021

@author: kjsanche

Description: 
A function to load the 5 minute granules from MODIS channel 1 
(0.65 microns) and the contrail mask for ML with a CNN.

To do:
ASAP:
-create blacklist of images to exclude (if images are to big)
-separate testing data
-plot testing data vs mask



lower priority:
-optimize image shape by transforming back to original satellite swath projection
-save and set up to load data as tfrecord
-organize/markdown/comment code


Input:
Path   (string)

        
        
Output:
MODISCh1 (2D uint32)
MASK     (2D uint16)
"""


from matplotlib import pyplot as plt
%matplotlib inline
import numpy as np
import struct
import os
import glob
import cv2
from UNET_Functions import unet_model, summary
from Sat_contrail_read import Extract_RawDef, extract_img, extract_mask, extract_imglist, get_model_memory_usage
from scipy.spatial import ConvexHull
from scipy.ndimage import rotate
from format_input import *
import tensorflow as tf

sys_details = tf.sysconfig.get_build_info()
print(sys_details)
cudnn_version = sys_details["cudnn_version"]
cuda_version = sys_details["cuda_version"]

print('cuda version: ', cuda_version)
print('cudNN version: ',cudnn_version)
print('TF version: ', tf.version.VERSION)


NEWMASK = False # use updated masks?
NCHANNELS=7 #number of channels + 1 (3, 7 or 8)
DVERSION = 'v1' #v1 = 8 channels, old masks; v2 = 3 channels, old mask; v3 = 8 channels, new mask; v4 = 3 channels, new mask



#tf.compat.v1.disable_eager_execution()
#tf.compat.v1.experimental.output_all_intermediates(True) 
path = os.getcwd()
proj_path = os.path.normpath(path + os.sep + os.pardir) #get parent directory
image_path = '/home/kjsanche/Desktop/ExternalSSD/SatContrailData/' #os.path.join(proj_path, 'data/')
save_TFrecord_path = '/home/kjsanche/Desktop/ExternalSSD/SatContrailData/TFrecords/' 


image0065, image0380, image0680, image0850, image1100, image1200, image1330, AUX_list, mask_list = extract_imglist(image_path, NEWMASK, NCHANNELS)

N = len(image1200)
print(N)
print(len(mask_list))

OrderedDict([('cpu_compiler', '/home/builder/ktietz/aggregate/tensorflow_recipes/ci_cpu/tensorflow-base_1614583966145/_build_env/bin/x86_64-conda_cos6-linux-gnu-gcc'), ('cuda_compute_capabilities', ['compute_35', 'compute_52', 'compute_60', 'compute_61', 'compute_70', 'compute_75']), ('cuda_version', '10.1'), ('cudnn_version', '7'), ('is_cuda_build', True), ('is_rocm_build', False)])
cuda version:  10.1
cudNN version:  7
TF version:  2.4.1
5980
5980


## extract image dimensions

In [2]:
dim = Extract_RawDef(AUX_list)

## make dataset

In [3]:
img0065_filenames = tf.constant(image0065)
img0380_filenames = tf.constant(image0380)
img0680_filenames = tf.constant(image0680)
img0850_filenames = tf.constant(image0850)
img1100_filenames = tf.constant(image1100)
img1200_filenames = tf.constant(image1200)
img1330_filenames = tf.constant(image1330)

masks_filenames = tf.constant(mask_list)
print(img1100_filenames.shape)
print(masks_filenames.shape)
dataset = tf.data.Dataset.from_tensor_slices((img0065_filenames, img0380_filenames, img0680_filenames, img0850_filenames, img1100_filenames, img1200_filenames, img1330_filenames, masks_filenames, dim))
#print(dataset)

(5980,)
(5980,)


In [4]:
def process_path(img0065_path, img0380_path, img0680_path, img0850_path, img1100_path, img1200_path, img1330_path, mask_path,dim):
# convert binary files to matrix of integers
    if NCHANNELS >=3:
        img5 = extract_img(str(img1100_path.numpy().decode('ascii')),int(dim[0].numpy()),int(dim[1].numpy()))
        img6 = extract_img(str(img1200_path.numpy().decode('ascii')),int(dim[0].numpy()),int(dim[1].numpy()))
        img = np.concatenate((img5, img6), axis=2)
    if NCHANNELS >=7:
        img2 = extract_img(str(img0380_path.numpy().decode('ascii')),int(dim[0].numpy()),int(dim[1].numpy()))
        img3 = extract_img(str(img0680_path.numpy().decode('ascii')),int(dim[0].numpy()),int(dim[1].numpy()))
        img4 = extract_img(str(img0850_path.numpy().decode('ascii')),int(dim[0].numpy()),int(dim[1].numpy()))
        img7 = extract_img(str(img1330_path.numpy().decode('ascii')),int(dim[0].numpy()),int(dim[1].numpy()))
        img = np.concatenate((img2, img3, img4, img7, img), axis=2)
    if NCHANNELS ==8:
        img1 = extract_img(str(img0065_path.numpy().decode('ascii')),int(dim[0].numpy()),int(dim[1].numpy()))
        img = np.concatenate((img1, img), axis=2)



    #img = np.concatenate((img1, img2, img3, img4, img7, img5, img6), axis=2)
    
    
    if NEWMASK:
        with open(str(mask_path.numpy().decode('ascii')), 'rb') as f:
            mask = np.zeros((4096,4096,1), dtype = int)
            x = np.load(f)
            mask[0:x.shape[0], 0:x.shape[1], 0] += x
    else:
        mask = extract_mask(str(mask_path.numpy().decode('ascii')),int(dim[0].numpy()),int(dim[1].numpy()))
        
    return tf.convert_to_tensor(img,dtype=tf.float16), tf.convert_to_tensor(mask,dtype=tf.int8)

def _set_shapes(img, mask):
    img.set_shape([4096, 4096, NCHANNELS-1])
    mask.set_shape([4096, 4096, 1])

    return img, mask 

#print(dataset)
image_ds = dataset.map(lambda aa, bb, cc, dd, ee, ff, gg, hh, ii: tf.py_function(process_path, [aa, bb, cc, dd, ee, ff, gg, hh, ii], [tf.float16, tf.int8]))
image_ds = image_ds.map(_set_shapes)
print(image_ds)



<MapDataset shapes: ((4096, 4096, 6), (4096, 4096, 1)), types: (tf.float16, tf.int8)>


In [5]:
def display(display_list):
    plt.figure(figsize=(15, 15))

    title = ['Input Image', 'True Mask', 'Predicted Mask']

    for i in range(len(display_list)):
        plt.subplot(1, len(display_list), i+1)
        plt.title(title[i])
        print(i)
        print(display_list[i].shape)
        if i == 0:
            plt.imshow(np.float32(display_list[i][:,:,-3]-display_list[i][:,:,-2]), cmap = 'gray')
        else:
            plt.imshow(np.float32(1*display_list[i][:,:,0]), cmap = 'gray') 
        plt.axis('off')
    plt.show()

In [6]:
for image, mask in image_ds.take(5):
    sample_image, sample_mask = image, mask
print(sample_image.shape)
print(sample_mask.shape)


(4096, 4096, 6)
(4096, 4096, 1)


In [7]:
def rotate_img(image, angle):
    rotated_image =  rotate(np.float32(image), angle=angle, reshape=True, order=0)
    #zoom into minimum bounding rectangle
    if np.max(rotated_image)>0:
        y_min, y_max, x_min, x_max = [np.min(np.nonzero(rotated_image)[0][:]), np.max(np.nonzero(rotated_image)[0][:]), np.min(np.nonzero(rotated_image)[1][:]), np.max(np.nonzero(rotated_image)[1][:])]
        return rotated_image, y_min, y_max, x_min, x_max
    return rotated_image, np.nan, np.nan, np.nan, np.nan

In [8]:
#used to find range of values for each channel
def Rng_Extremes(x, rng):
    mx = np.max(np.max(x))
    x[x==0]=99999
    mn = np.min(np.min(x))
    x[x==99999]=0
    if rng == []:
        return [mn, mx]
    if mn<rng[0]: 
        mn = mn
    else:
        mn = rng[0]
    if mx>rng[1]: 
        mx = mx
    else:
        mx = rng[1]
    return [mn, mx]
        
c1rng =[]
c2rng =[]
c3rng =[]
c4rng =[]
c5rng =[]
c6rng =[]
c7rng =[]

In [ ]:



fileList=tf.io.gfile.glob([save_TFrecord_path + '*' + DVERSION +'.tfrecords'])
cnt = 0

for image, mask in image_ds:
    sample_image, sample_mask = image, mask
    filename = mask_list[cnt][-36:-14] + DVERSION
    if NEWMASK:
        filename=mask_list[cnt][-36-len('update'):-14-len('update')] + DVERSION #different names for cases with updatedmask
    print(filename)
    cnt+=1
    
    #skip file if file already exists or nans/inf values are in images
    if save_TFrecord_path+'1_1' + filename + '.tfrecords' in fileList:
        continue

    if np.any(np.isnan(sample_image)) or np.any(np.isinf(sample_image)):
        #seems some values in sample_image[:,:,0] are inf sometimes
        continue #sys.exit()

    if np.any(np.isnan(sample_mask)) or np.any(np.isinf(sample_mask)):
        #print('mask')
        continue #sys.exit()

        
        
        
    #find coordinates of all nonzero coordinates
    nonzero =np.asarray(np.nonzero(np.float32(sample_image[:,:,-2]))).T
    if len(nonzero)==0:
        #at least 1 case is all zeros and breaks code
        continue

    #extract convexhull coordinates, first point and last point must be the same. 
    hull = ConvexHull(nonzero)
    hull = np.vstack((nonzero[hull.vertices,0],nonzero[hull.vertices,1])).T
    hull = np.vstack((hull,hull[0,:]))

    #calculate angle of rotation for minimum bounding rectangle
    angle, barea, bwidth, bheight, center_point, corner_points = minBoundingRect(hull)
    
    rotated_image, y_min, y_max, x_min, x_max = rotate_img(sample_image, angle=90-angle*180/np.pi)
    rotated_labels, _, _, _, _ = rotate_img(sample_mask, angle=90-angle*180/np.pi)
    
    w = x_max-x_min
    h = y_max-y_min
    
    #determine min/max dimensions
    maxCaseDim = np.max([w,h])
    minCaseDim = np.min([w,h])

    #rotate 90 degrees to make width the bigger side if needed
    if h > w:
        rotated_image, y_min, y_max, x_min, x_max = rotate_img(rotated_image, angle=90)
        rotated_image_zoom = rotated_image[y_min:y_max,x_min:x_max]
        rotated_labels, _, _, _, _ = rotate_img(rotated_labels, angle=90)
        rotated_labels_zoom = rotated_labels[y_min:y_max,x_min:x_max]
    else:
        rotated_image_zoom = rotated_image[y_min:y_max,x_min:x_max]
        rotated_labels_zoom = rotated_labels[y_min:y_max,x_min:x_max]
 
    #make images and mask have same h x w (2048,4096)
    mask = np.zeros((2048,4096,1), dtype = float)
    mask[0:rotated_labels_zoom.shape[0], 0:rotated_labels_zoom.shape[1]] += rotated_labels_zoom


    img_test = np.zeros((2048,4096,NCHANNELS), dtype = float)
    img_test[0:rotated_image_zoom.shape[0], 0:rotated_image_zoom.shape[1], :NCHANNELS-1] += rotated_image_zoom[:,:,1-NCHANNELS:]
    # set last channel (valid channel) to 1 for the pixels containing part of the satellite image (keep 0 for the blank pixels)
    img_test[img_test[:,:,-2] > 0 ,-1] = 1 

    #check min and max values of channels
    '''
    c1rng = Rng_Extremes(img_test[:,:,0],c1rng)
    c2rng = Rng_Extremes(img_test[:,:,1],c2rng)
    c3rng = Rng_Extremes(img_test[:,:,2],c3rng)
    c4rng = Rng_Extremes(img_test[:,:,3],c4rng)
    c5rng = Rng_Extremes(img_test[:,:,4],c5rng)
    c6rng = Rng_Extremes(img_test[:,:,5],c6rng)
    c7rng = Rng_Extremes(img_test[:,:,6],c7rng)
    '''
    
    
    
    

    img_test = np.float64(img_test)
    #resize images to decreased resolution
    img_test = cv2.resize(img_test, dsize=(1024, 512), interpolation=cv2.INTER_NEAREST)
    mask = cv2.resize(mask, dsize=(1024, 512), interpolation=cv2.INTER_NEAREST)

    #add missing dimensions
    img_test = np.expand_dims(img_test, axis=0)
    mask = np.expand_dims(mask, axis=0)

    #set zeros to match min value in pixel range (min value is approximately 0 in first three channels)
    if NCHANNELS-5>0:
        tmp = img_test[:,:,:,NCHANNELS-5:]
        tmp[tmp==0]=17000
        img_test[:,:,:,NCHANNELS-5:]=tmp
        BT_rng = np.float64([17000, 34000]) #this range is good for 4 upper channels. assuming range of 170-340 K (BT is multiplied by 100)
        img_test[:,:,:,NCHANNELS-5:-1] = (img_test[:,:,:,NCHANNELS-5:-1]-np.mean(BT_rng)) / ((BT_rng[1]-BT_rng[0])/2) 
        BT_rng = np.float64([0, 66535]) #lower 3 channels have a much larger range.
        img_test[:,:,:,:NCHANNELS-5] = (img_test[:,:,:,:NCHANNELS-5]-np.mean(BT_rng)) / ((BT_rng[1]-BT_rng[0])/2)
    else:
        img_test[img_test==0]=17000
        BT_rng = np.float64([17000, 34000]) #this range is good for 4 upper channels. assuming range of 170-340 K (BT is multiplied by 100)
        img_test[:,:,:,:-1] = (img_test[:,:,:,:-1]-np.mean(BT_rng)) / ((BT_rng[1]-BT_rng[0])/2) 

    
    #normalize img based on channel ranges and set mask to binary values identifying contrails
    mask[mask>0.5]=1
    mask[mask<=0.5]=0


            
    
    #change type to decrease size
    img_test = np.float16(img_test)
    mask = np.int8(mask)

    write_images_to_tfr_long(img_test, mask, filename=filename, max_files=1, out_dir=save_TFrecord_path)

    



MYD021KM-A2018100.0350v1
MYD021KM-A2018100.0845v1
MYD021KM-A2018100.0710v1
MYD021KM-A2018100.2345v1
MYD021KM-A2018100.0605v1
MYD021KM-A2018100.1040v1
MYD021KM-A2018100.0700v1
MYD021KM-A2018100.2015v1
MYD021KM-A2018100.1200v1
MYD021KM-A2018100.0025v1
MYD021KM-A2018100.1350v1
MYD021KM-A2018100.1900v1
MYD021KM-A2018100.1730v1
MYD021KM-A2018100.1340v1
MYD021KM-A2018100.1835v1
MYD021KM-A2018100.1405v1
MYD021KM-A2018100.1650v1
MYD021KM-A2018100.0205v1
MYD021KM-A2018100.0235v1
MYD021KM-A2018100.1850v1
MYD021KM-A2018100.0745v1
MYD021KM-A2018100.0110v1
MYD021KM-A2018100.1055v1
MYD021KM-A2018100.1030v1
MYD021KM-A2018100.0855v1
MYD021KM-A2018100.0030v1
MYD021KM-A2018100.2230v1
MYD021KM-A2018100.0055v1
MYD021KM-A2018100.0105v1
MYD021KM-A2018100.0220v1
MYD021KM-A2018100.0840v1
MYD021KM-A2018100.1400v1
MYD021KM-A2018100.1910v1
MYD021KM-A2018100.0410v1
MYD021KM-A2018100.1825v1
MYD021KM-A2018100.1010v1
MYD021KM-A2018100.1335v1
MYD021KM-A2018100.0355v1
MYD021KM-A2018100.0705v1
MYD021KM-A2018100.2330v1


100%|██████████| 1/1 [00:00<00:00, 47.21it/s]


Wrote 1 elements to TFRecord


MYD021KM-A2018100.0035v1

Using 1 shard(s) for 1 files, with up to 1 samples per shard


100%|██████████| 1/1 [00:00<00:00, 55.72it/s]

...


Wrote 1 elements to TFRecord


MYD021KM-A2018097.2155v1

Using 1 shard(s) for 1 files, with up to 1 samples per shard


100%|██████████| 1/1 [00:00<00:00, 58.01it/s]


Wrote 1 elements to TFRecord


MYD021KM-A2018097.2135v1


In [ ]:
x = [[[2,3], [4,5]]]
print(x)

In [ ]:
print(np.shape(x))
y= np.concatenate((x, x), axis=2)
z= np.concatenate((x, y), axis=2)
print(np.shape(y))
print(np.shape(z))

In [ ]:
def display(display_list):
    plt.figure(figsize=(15, 15))

    title = ['Input Image', 'True Mask', 'Predicted Mask']

    for i in range(len(display_list)):
        plt.subplot(1, len(display_list), i+1)
        plt.title(title[i])
        print(i)
        print(display_list[i].shape)
        if i == 0:
            plt.imshow(np.float32(display_list[i][0,:,:,-3]-display_list[i][0,:,:,-2]))
        else:
            plt.imshow(np.float32(1*display_list[i][0,:,:]))
        plt.axis('off')
    plt.show()

print(img_test.shape)
print(mask.shape)
#mask[img_test[:,:,:,0] == 0] = 2
#test = img_test[:,:,:,0] < .1
print(img_test)
display([img_test,mask])

